In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
pal = sns.color_palette("dark")
cpal = sns.cubehelix_palette(start=-0.25, rot=2, as_cmap=True)

In [3]:
source_path = Path(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2024_11_04\recon2\backtracking")
source1 = tifffile.imread(source_path / "recon_fused_tp_180_ch_0.tif")
source2 = tifffile.imread(source_path / "recon_fused_tp_181_ch_0.tif")

In [63]:
print(source1.shape)

(365, 891, 354)


In [10]:
from skimage.filters import difference_of_gaussians

dog1 = difference_of_gaussians(source1, 2, 6)

In [66]:
spots_df = pd.read_csv(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2024_11_04\recon2\dis3\out\tracked_spots.csv")
example_points = spots_df[spots_df["FRAME"] == 181]
example_points = example_points[example_points["track_id"] != 0]
X = example_points[["POSITION_X", "POSITION_Y", "POSITION_Z"]].values

In [16]:
import napari
from scipy.ndimage import sobel

viewer = napari.Viewer()

s = np.zeros_like(source1)

for axis in range(3):
    s += sobel(-dog1, axis=axis)**2
viewer.add_image(np.sqrt(s), name="source1_sobel", colormap="magenta")
viewer.add_image(source1, name="source1", colormap="green")
napari.run()

In [55]:
from skimage.feature import peak_local_max
from scipy.ndimage import distance_transform_edt
from skimage import segmentation

peaks = peak_local_max(dog1, min_distance=2, threshold_abs=20)
img = np.zeros(source1.shape)
img[peaks[:, 0], peaks[:, 1], peaks[:, 2]] = np.arange(1, len(peaks) + 1)
dist = distance_transform_edt(img == 0)
w = segmentation.watershed(-dog1, img.astype(int), mask=dist < 18)

In [75]:
np.array(list(peaks_map.values()))

array([[116, 183,  67],
       [270, 161, 251],
       [289, 203, 247],
       ...,
       [108, 819, 166],
       [105, 304,  99],
       [  0,   0,   0]], dtype=int64)

In [67]:
peaks_map = {i + 1: p for i, p in enumerate(peaks)}
peaks_map[0] = (0, 0, 0)
example_points["new_label"] = [w[tuple(p.astype(int))] for p in X]
example_points["new_x"] = example_points["new_label"].map(lambda x: peaks_map[x][0])
example_points["new_y"] = example_points["new_label"].map(lambda x: peaks_map[x][1])
example_points["new_z"] = example_points["new_label"].map(lambda x: peaks_map[x][2])
X_new = example_points[["new_x", "new_y", "new_z"]].values

In [62]:
print(np.max(X, axis=0))

[332. 874. 343.]


In [61]:
print(peaks_map[150])
print(np.argwhere(img == 150))

[320 329 121]
[[320 329 121]]


In [68]:
print(sum(example_points["new_label"] == 0))
example_points

0


,Unnamed: 0,ID,VISIBILITY,RADIUS,QUALITY,POSITION_T,POSITION_X,POSITION_Y,FRAME,POSITION_Z,track_id,position_cluster,distance_from_edge,tracklet_id,parent_id,daughter_id,new_label,new_x,new_y,new_z
375311,422574,422574,1.0,1.462862,811.326660,0.0,269.0,358.0,181.0,56.0,139,0,70.691565,20865,419959,425548,50,269,358,57
375312,422575,422575,1.0,1.462862,805.188960,0.0,324.0,333.0,181.0,236.0,258,0,12.329924,15467,419919,425705,12,324,333,235
375313,422576,422582,1.0,1.462862,757.105800,0.0,133.0,301.0,181.0,41.0,169,0,114.000000,20242,419925,425604,16,134,301,41
375314,422577,422583,1.0,1.462862,757.056400,0.0,321.0,355.0,181.0,245.0,127,0,18.905884,20341,419916,425590,9,321,355,245
375315,422578,422580,1.0,1.462862,759.925230,0.0,81.0,383.0,181.0,69.0,167,0,62.000000,20955,419949,425550,40,81,382,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377782,425524,425522,1.0,1.462862,50.353150,0.0,286.0,453.0,181.0,258.0,1518,0,59.761961,18176,422339,428464,2639,287,452,257
377783,425526,425520,1.0,1.462862,50.371950,0.0,255.0,318.0,181.0,273.0,24211,0,77.892621,24721,0,428638,3232,255,318,273
377784,425535,425529,1.0,1.462862,50.230713,0.0,248.0,362.0,181.0,74.0,5324,0,92.063433,23389,419756,428493,3223,249,362,74
377785,425538,425540,1.0,1.462862,50.060013,0.0,77.0,137.0,181.0,142.0,8905,0,44.466842,24737,0,428654,3286,76,138,142


In [70]:
viewer = napari.Viewer()
viewer.add_image(source1)
viewer.add_image(img, name="peaks")
viewer.add_image(w, name="watershed")
viewer.add_points(X[:], size=5, face_color="white", out_of_slice_display=True)
viewer.add_points(X_new[:], size=5, face_color="red", out_of_slice_display=True)
napari.run()